In [59]:
from langchain_huggingface import HuggingFacePipeline
from langchain_core.messages import HumanMessage,AIMessage
from transformers import AutoTokenizer,AutoModelForCausalLM,pipeline
print("Loading the model.")
model="microsoft/DialoGPT-small"
tokenize=AutoTokenizer.from_pretrained(model)
model2=AutoModelForCausalLM.from_pretrained(model)

Loading the model.


In [60]:
#pipelining
pipe=pipeline(
    "text-generation",
    model=model2,
    tokenizer=tokenize,
    max_length=200,
    temperature=0.6,
    do_sample=True,)

Device set to use cpu


In [61]:
#using hugging face
llm=HuggingFacePipeline(pipeline=pipe)

In [62]:
def prompt(user_input, history):
    if not history:
        return user_input
    context=""
    recent_history=history[-6:] if len(history)>6 else history
    for msg in recent_history:
        if isinstance(msg, HumanMessage):
            context+=f"Human: {msg.content}\n"
        elif isinstance(msg, AIMessage):
            context+=f"Assistant: {msg.content}\n"
    return f"{context}Human: {user_input}\nAssistant:"
def clean(response):
    lines=response.split('\n')
    for line in lines:
        if line.startswith('Assistant:') and len(line) > 11:
            return line[11:].strip()
    return response.strip()

In [63]:
print("Chatbot ready to go type 'quit' to exit or 'clear' to clear history.")
while True:
    user_input=input("\nYou: ").strip()
    if user_input.lower()=='quit':
        print("Goodbye!")
        break
    elif user_input.lower()=='clear':
        chat_history=[]
        print("Cleared history")
        continue
    elif not user_input:
        continue
    try:
        prompt=prompt(user_input, chat_history)
        response=llm.invoke(prompt)
        bot_response=clean(response)
        chat_history.append(HumanMessage(content=user_input))
        chat_history.append(AIMessage(content=bot_response))
        print(f"Bot: {bot_response}")
    except Exception as e:
        print(f"Error:{e}")
        print("Bot:Sorry this cannot be process.")

Chatbot ready to go type 'quit' to exit or 'clear' to clear history.



You:  hello


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Bot: hello??



You:  quit


Goodbye!
